# 02. Dijkstra SSSP Benchmark\n\nThis notebook benchmarks variants of Dijkstra's algorithm for SSSP. It assumes non-negative edge weights.\n- `dijkstra_serial`\n- `dijkstra_openmp`\n- `dijkstra_cuda`\n- `dijkstra_hybrid`

## 1. Setup\n\nThis cell ensures the notebook is running in the correct project root directory and copies the shared utility functions from the setup notebook.

In [ ]:
import os\nif os.path.basename(os.getcwd()) == 'notebooks':\n    %cd ..\nprint(f'Working directory: {os.getcwd()}')\n!echo '\n== bin/ contents =='; ls -F bin/\n\nimport subprocess, statistics, re, os, json, time, pandas as pd\n\ndef run_command(cmd, timeout=300):\n    try:\n        print(f'  > {cmd}')\n        return subprocess.run(cmd, shell=True, capture_output=True, text=True, check=True, timeout=timeout).stdout\n    except subprocess.CalledProcessError as e:\n        print(f'    stderr: {e.stderr.strip()}')\n    except subprocess.TimeoutExpired:\n        print('    timeout')\n    return None\n\ndef parse_time(out):\n    if not out: return None\n    m = re.search(r'time:\\s*([0-9]*\\.?[0-9]+)\\s*(ms|s|sec|seconds)?', out, re.I)\n    if not m: return None\n    val = float(m.group(1)); unit = (m.group(2) or 's').lower()\n    return val/1000.0 if unit.startswith('ms') else val\n\ndef time_exe(cmd, warmups=1, runs=3):\n    if not cmd: return None\n    for _ in range(warmups): _ = run_command(cmd)\n    samples = []\n    for _ in range(runs):\n        t = parse_time(run_command(cmd))\n        if t is not None: samples.append(t)\n    return statistics.median(samples) if samples else None

## 2. Dataset Selection

In [ ]:
#@markdown **Dataset Selection** (set `use_real_data` to toggle)\nuse_real_data = False  #@param {type:'boolean'}\nreal_graph_url = 'https://raw.githubusercontent.com/networkrepository/NRG/edge_list/small_graph.edgelist'\n\nif use_real_data:\n    # (Code for downloading real data - omitted for brevity, assumed to be similar to BF notebook)\n    print('Using real dataset for Dijkstra.')\nelse:\n    print('Using synthetic random graphs for Dijkstra.')

## 3. Benchmark Parameters

In [ ]:
#@markdown ### Benchmark Parameters (Dijkstra SSSP)\nV_list = '500,1000,2000,5000'  #@param {type:'string'}\nmin_w = 1                      #@param {type:'integer'}\nmax_w = 100                    #@param {type:'integer'}\ndensity = 0.1                  #@param {type:'number'}\nthreads = 8                    #@param {type:'integer'}\ndelta_value = 5                #@param {type:'integer'}\n\nV_list = [int(x) for x in V_list.split(',')]executables = ['dijkstra_serial', 'dijkstra_openmp', 'dijkstra_cuda', 'dijkstra_hybrid']

### Algorithmic Variant: Δ-Stepping\n\nThe Δ-Stepping algorithm parallelizes Dijkstra by processing vertices in batches. It relaxes 'light' edges (weight < Δ) synchronously within a bucket, and 'heavy' edges asynchronously. This balances load and reduces synchronization overhead, making it effective for parallel execution.

## 4. Command Builder

In [ ]:
def build_cmd_dijkstra(exe, v, *, min_w, max_w, density, threads, delta):\n    path = os.path.join('bin', exe)\n    if not os.path.exists(path): return None\n    # Base arguments for all Dijkstra variants\n    args = [str(v), str(min_w), str(max_w), str(density)]\n    \n    # Add threads for OpenMP/Hybrid, delta for those that use it\n    if 'openmp' in exe or 'hybrid' in exe:\n        args.append(str(threads))\n        # Assuming delta-stepping might be used in parallel versions\n        # This part may need adjustment based on specific exe logic\n        if 'delta' in exe: # Hypothetical name\n             args.append(str(delta))\n            \n    return f'{path} {' '.join(args)}'

## 5. Run Benchmarks

In [ ]:
rows = []\nfor v in V_list:\n    print(f'\nDijkstra for V={v}')\n    row = {'vertices': v}\n    for exe in executables:\n        cmd = build_cmd_dijkstra(exe, v, min_w=min_w, max_w=max_w, density=density, threads=threads, delta=delta_value)\n        t = time_exe(cmd)\n        row[exe] = t\n        if t is not None: print(f'  {exe}: {t:.6f}s')\n    rows.append(row)\n\ndf_dijkstra = pd.DataFrame(rows).set_index('vertices').sort_index()\ndf_dijkstra.to_csv('dijkstra_times.csv')\ndisplay(df_dijkstra)

## 6. Speedup Analysis

In [ ]:
import numpy as np, seaborn as sns, matplotlib.pyplot as plt\nbase = df_dijkstra['dijkstra_serial']\nspeed = pd.DataFrame({\n    'dijkstra_openmp_speedup': base / df_dijkstra['dijkstra_openmp'],\n    'dijkstra_cuda_speedup':   base / df_dijkstra['dijkstra_cuda'],\n    'dijkstra_hybrid_speedup': base / df_dijkstra['dijkstra_hybrid']\n}, index=df_dijkstra.index)\n\ndisplay(speed)\nsns.lineplot(data=speed.reset_index().melt('vertices', var_name='variant', value_name='speedup'),\n             x='vertices', y='speedup', hue='variant', marker='o')\nplt.axhline(1, ls='--', c='gray'); plt.yscale('log'); plt.show()\nspeed.to_csv('dijkstra_speedup.csv')

## 7. Profiling with Nsight Systems\n\nDemonstration of how to profile the CUDA implementation using Nsight Systems to visualize the execution timeline.

In [ ]:
v_profile = 2000\nprofile_cmd = f'nsys profile -o dijkstra_profile --force-overwrite true bin/dijkstra_cuda {v_profile} {min_w} {max_w} {density}'\nprint('Running Nsight Systems profiler... (this may take a moment)')\noutput = run_command(profile_cmd)\nif output:\n    print('Profiling complete. Report saved to dijkstra_profile.nsys-rep')\n    print(output)